<a href="https://colab.research.google.com/github/asmaael33/datanalyzer/blob/master/TestWithGoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pyyaml h5py  # Required to save models in HDF5 format
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.3.0


In [5]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [7]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.2136 - accuracy: 0.6596
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 1.1455 - accuracy: 0.6790 - val_loss: 0.7013 - val_accuracy: 0.7900
Epoch 2/10
31/32 [============================>.] - ETA: 0s - loss: 0.4010 - accuracy: 0.8891
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.4115 - accuracy: 0.8860 - val_loss: 0.5063 - val_accuracy: 0.8440
Epoch 3/10
27/32 [========================>.....] - ETA: 0s - loss: 0.2817 - accuracy: 0.9259
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.2814 - accuracy: 0.9240 - val_loss: 0.4548 - val_accuracy: 0.8610
Epoch 4/10
31/32 [============================>.] - ETA: 0s - loss: 0.2001 - accuracy: 0.9567
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 

In [8]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [9]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_1/cp.ckpt'

In [11]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4058 - accuracy: 0.8700
Restored model, accuracy: 87.00%


In [12]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4058 - accuracy: 0.8700
Restored model, accuracy: 87.00%


In [13]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1703 - accuracy: 0.6740
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4083 - accuracy: 0.8880
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2753 - accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1984 - accuracy: 0.9520
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1467 - accuracy: 0.9660
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [14]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	saved_model.pb	variables


In [15]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4254 - accuracy: 0.0870
Restored model, accuracy:  8.70%
(1000, 10)


In [17]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 0s 6ms/step - loss: 1.1827 - accuracy: 0.6510
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4336 - accuracy: 0.8730
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2899 - accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2018 - accuracy: 0.9540
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1522 - accuracy: 0.9730


In [18]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4170 - accuracy: 0.0900
Restored model, accuracy:  9.00%
